# Tutorial Notebook for IBoat - PMCTS

<div class="alert-danger">
*Please be aware that this tutorials shall not replace the project thorought documentation that you can find at :
https://pbarde.github.io/IBoatPIE/ * 
</div>

<div class="alert-success"> After completing this tutorial you should be able to use all the tools devellopped during the IBoat - PMCTS projet. Using these tools you should investigate different algorithms tunings to find an optimal strategy and compare it to the isochrones method. <br/>
    
    
** Finally, all the code of this notebook can be found in the following [Tutorial script](Tutorial.py) and will run much more smoothly than in this jupyter notebook**
</div>

<div class="alert-danger">
    **Before starting :** why don't you create a conda environment to make sure everything plays smoothly ? <br/>
Install Anaconda 3 ([Anaconda Download](https://www.anaconda.com/download/#linux)) if not done already and run the following commands in your terminal (linux of course).
</div>
```
conda create --name mcts
source activate mcts
conda install pip
conda install jupyter
conda install basemap
conda install netcdf4
```

*Then, when you are finished playing you can just remove the conda environment with the line :*
```
source deactivate
conda remove --name mcts --all
```

## Weather forecasts and Simulators

In this section you'll learn how to download, load, process and visualize weather forecasts. We will also create the simulators that are used in the PMCTS.

### Downloading weather forecasts

Let's start by downloading some forecasts. 
<div class="alert-warning">
*You'll need to change the* `mydate` *variable* </div>

In [1]:
import forest as ft

# The starting day of the forecast. If it's too ancient, the forecast might not be available anymore
mydate = '20180228' # for February 2, 2018

# We will download the mean scenario (id=0) and the first 2 perturbed scenarios
scenario_ids = range(3)
ft.download_scenarios(mydate, latBound=[40, 50], lonBound=[-15 + 360, 360], scenario_ids=scenario_ids)

Saved into : ../data/20180228_0000z.obj
Saved into : ../data/20180228_0100z.obj
Saved into : ../data/20180228_0200z.obj


### Loading weather objects


Now that we have downaloaded some forecasts, let's load and create simulators out of them.  

In [1]:
import matplotlib
matplotlib.use('qt5Agg')
import matplotlib.pyplot as plt
import forest as ft
# The starting day of the forecast. If it's too ancient, the forecast might not be available anymore
mydate = '20180228' # for February 2, 2018

# We will download the mean scenario (id=0) and the first 2 perturbed scenarios
scenario_ids = range(3)
Weathers = ft.load_scenarios(mydate, latBound=[40, 50], lonBound=[-15 + 360, 360], scenario_ids=scenario_ids)

Loaded : ../data/20180228_0000z.obj
Loaded : ../data/20180228_0100z.obj
Loaded : ../data/20180228_0200z.obj


### Create simulators and display wind conditions

We define the main parameters of our simulators, we create them and we visualize the corresponding wind conditions. The plots are interactives (use the upper-left icons to navigate through the weather forecast).
<div class="alert-warning">
*Our code is not supposed to be executed in a jupyter notebook which is a bit capricious and does not handle mutli-processing properly and animations. So you have to run multiple cells in order to visualize multiple scenaris. To visualize multiple scenarios simulataneously from you should use in a script* `ft.play_multiple_scenarios(Sims)`
</div>

In [9]:
%%capture
from multiprocessing import Process

NUMBER_OF_SIM = 3  # <=20 
SIM_TIME_STEP = 6  # in hours
STATE_INIT = [0, 44, 355]
N_DAYS_SIM = 3  # time horizon in days

Sims = ft.create_simulators(Weathers, numberofsim=NUMBER_OF_SIM, simtimestep=SIM_TIME_STEP,
                            stateinit=STATE_INIT, ndaysim=N_DAYS_SIM)        

# in the notebook we can visualize scenarios only one by one. 
Sims[0].play_scenario()
    
## /!\ if executing from a .py script, you better use this to have multiple interactive plots: 
# ft.play_multiple_scenarios(Sims)

In [10]:
%%capture
Sims[1].play_scenario(1)

In [11]:
%%capture
Sims[2].play_scenario(2)

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))



*If you are frustrated by the interactive plot you are invited to copy paste the code below in a .py file and replace* 
```python
for ii, sim in enumerate(Sims) : 
    sim.play_scenario(ii)
```
*by*
```python
ft.play_multiple_scenarios(Sims)
```
*execute your new .py file and enjoy.*


## The Parallel Monte-Carlo Tree Search

In this section we will see how to launch a PMCTS and visualize the results.

### Initialisation of the search

First of all we define a departure point, a mission heading and we compute the corresponding destination point and reference travel time. And we visualize the mean trajectories per scenarios during the two initialization phases. 

In [12]:
%%capture
import matplotlib
matplotlib.rcParams.update({'font.size': 10})
missionheading = 0 # direction wrt. true North we want to go the furthest.
ntra = 50 # number of trajectories used during the initialization
destination, timemin = ft.initialize_simulators(Sims, ntra, STATE_INIT, missionheading, plot=True)
plt.show()

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [5]:
print("destination : {} & timemin : {}".format(destination, timemin))

destination : [47.12514559106446, 355.0] & timemin : 2.22523753998074


### Create a Forest and launch a PMCTS

Now we create a Forest (the object managing the worker trees and the master tree) and we launch a search.

In [13]:
budget = 100 # number of nodes we want to expand in each worker
frequency = 10 # number of steps performed by worker before writing the results into the master
forest = ft.Forest(listsimulators=Sims, destination=destination, timemin=timemin, budget=budget)

if __name__ == '__main__':
    master_nodes = forest.launch_search(STATE_INIT, frequency)


Iteration 50 on 100 for workers 1

Iteration 50 on 100 for workers 2

Iteration 50 on 100 for workers 0

Iteration 100 on 100 for workers 1

Iteration 100 on 100 for workers 2

Iteration 100 on 100 for workers 0



Since the `master_nodes` object was created has a memory shared by multiple processes we need to do a deep copy of it before processing it. 

In [14]:
from master_node import deepcopy_dict
new_dict = deepcopy_dict(master_nodes)

At this point we can create a `MasterTree` object to process the results and get the optimal policies. We usually add it to the forest object. 

In [15]:
from master import MasterTree
forest.master = MasterTree(Sims, destination, nodes=new_dict)
forest.master.get_best_policy()

Global policy
Depth 1: best reward = 0.18861289173789172 for action = 0
Depth 2: best reward = 0.22916666666666666 for action = 180
Depth 3: best reward = 0.2506210205196431 for action = 180
Depth 4: best reward = 0.313121020519643 for action = 0
Policy for scenario 0
Depth 1: best reward = 0.3125 for action = 315
Depth 2: best reward = 0.4375 for action = 90
Depth 3: best reward = 0.3125 for action = 0
Policy for scenario 1
Depth 1: best reward = 0.13194444444444445 for action = 0
Depth 2: best reward = 0.3125 for action = 180
Depth 3: best reward = 0.3125 for action = 0
Policy for scenario 2
Depth 1: best reward = 0.26358695652173914 for action = 0
Depth 2: best reward = 0.3482142857142857 for action = 0
Depth 3: best reward = 0.625 for action = 90
Depth 4: best reward = 0.3125 for action = 270


It is also possible to plot the global uct tree colored with nodes colored by utility, exploration and exploitation values.

In [16]:
%%capture
forest.master.plot_tree_uct();
forest.master.plot_tree_uct(1);

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


We can also follow the global best policy and look at the reward distribution along this path.

In [18]:
%%capture
forest.master.plot_hist_best_policy(interactive = True)

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


Finally we can save the results. 

In [19]:
forest.master.save_tree("my_tuto_results")

## Isochrones and Validation